**Using only players considered in mvp voting**

In [10]:
import pandas as pd

In [11]:
stats = pd.read_csv("player_mvp_stats.csv")

In [12]:
del stats["Unnamed: 0"]

In [13]:
pd.isnull(stats).sum()
stats_all_players = stats.fillna(0)
stats_all_players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,0.477,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,0.455,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,0.340,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,0.484,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
15232,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,0.286,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
15233,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,0.470,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45
15234,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,0.459,...,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.0,103.6,103.8,-0.45


In [14]:
mvp_players = stats_all_players[stats_all_players["Share"] > 0]
mvp_players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
6,Magic Johnson,PG,31,LAL,79,79,37.1,5.9,12.4,0.477,...,960.0,0.518,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
42,Charles Barkley,PF,30,PHO,65,65,35.4,8.0,16.1,0.495,...,1010.0,0.005,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
50,Kevin Johnson,PG,27,PHO,67,67,36.6,7.1,14.6,0.487,...,1010.0,0.001,Phoenix Suns,56,26,0.683,7.0,108.2,103.4,4.68
58,Charles Barkley,PF,31,PHO,68,66,35.0,8.1,16.8,0.486,...,1050.0,0.091,Phoenix Suns,59,23,0.720,0.0,110.6,106.8,3.86
71,Charles Barkley,PF,32,PHO,71,71,37.1,8.2,16.3,0.500,...,1130.0,0.007,Phoenix Suns,41,41,0.500,23.0,104.3,104.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15073,Shaquille O'Neal,C,26,LAL,49,49,34.8,10.4,18.1,0.576,...,1180.0,0.075,Los Angeles Lakers,31,19,0.620,4.0,99.0,96.0,2.68
15106,Jalen Brunson,PG,26,NYK,68,68,35.0,8.6,17.6,0.491,...,1000.0,0.001,New York Knicks,47,35,0.573,10.0,116.0,113.1,2.99
15146,Dominique Wilkins,SF,31,ATL,81,81,38.0,9.5,20.2,0.470,...,960.0,0.030,Atlanta Hawks,43,39,0.524,18.0,109.8,109.0,0.72
15162,Jason Kidd,PG,21,DAL,79,79,33.8,4.2,10.8,0.385,...,1050.0,0.007,Dallas Mavericks,36,46,0.439,26.0,103.2,106.1,-2.39


**Modeling**

**Train/Test Split**

In [15]:
def train_test_split(year, df):
    predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']
    #test year = selected year, train year = other years outside of selected year
    train = df[df['Year'] < year]
    test = df[df['Year'] == year]
    
    X_train = train[predictors]
    X_test = train["Share"]

    Y = test[predictors]
    
    return X_train, X_test, Y, test

**Function to run multiple models**

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

In [48]:
def run_model(regressor, X_train, X_test, Y, test):
    model = regressor
    model.fit(X_train, X_test) 
    predictions = model.predict(Y)
    
    predictions = pd.DataFrame(predictions, columns=["Predictions"], index = test.index)
    combinations = pd.concat([test[["Player", "Share"]], predictions], axis = 1)
    combinations.sort_values("Predictions", ascending = False)
    combinations = combinations.sort_values("Share", ascending = False)
    combinations["Rank"] = list(range(1, combinations.shape[0] + 1))
    combinations = combinations.sort_values("Predictions", ascending = False)
    combinations["Predicted_Rank"] = list(range(1, combinations.shape[0] + 1))

    
    # mae = mean_absolute_error(combinations["Share"], combinations["Predictions"])
    # rmse = np.sqrt(mean_squared_error(combinations["Share"], combinations["Predictions"]))
    return combinations

**Find avg metric score across all seasons**

In [45]:
# years = [year for year in range(1992, 2024)]

# def run_model_average(df, regressor, print_metric = False):
#     mae_lst = []
#     r2_lst = []
    
#     for year in (years):
#         X_train, X_test, Y, test = train_test_split(year=year, df=df)
#         mvp, mae, r2, = run_model(regressor,
#                                  X_train, 
#                                  X_test, 
#                                  Y, 
#                                  test)
                                        
#         mae_lst.append(mae)
#         r2_lst.append(r2)
        
#     d = {
#     'year': years,
#     'MAE': mae_lst,
#     'R squared': r2_lst,
#     }

#     summary = pd.DataFrame(d)
#     avg_mae = summary['MAE'].mean()
#     avg_r2  = summary['R squared'].mean()
    
#     if print_metric == True:
#         print(f"Average MAE: {avg_mae}")
#         print(f"Average R squared: {avg_r2}")

#     return summary

**Setting year for train/test split**

In [46]:
X_train, X_test, Y, test = train_test_split(2011, mvp_players)

**Linear Regression Model**

In [49]:
mvp_linear_race = run_model(LinearRegression(), X_train=X_train, X_test=X_test, Y = Y, test=test)
mvp_linear_race

,Player,Share,Predictions,Rank,Predicted_Rank
14083,LeBron James,0.431,0.440471,3,1
13220,Derrick Rose,0.977,0.436848,1,2
7312,Kobe Bryant,0.354,0.368741,4,3
14075,Dwyane Wade,0.020,0.292347,7,4
12594,Dwight Howard,0.531,0.250367,2,5
13585,Kevin Durant,0.157,0.249690,5,6
12981,Dirk Nowitzki,0.093,0.210125,6,7
10006,Manu Ginóbili,0.017,0.139454,8,8
10012,Tony Parker,0.002,0.119975,13,9
10471,Rajon Rondo,0.004,0.042493,11,10


**Random Forest Regressor Model**

In [38]:
mvp_race_rf = run_model(RandomForestRegressor(), X_train, X_test, Y, test)
mvp_race_rf

,Player,Share,Predictions,Rank,Predicted_Rank
14083,LeBron James,0.431,0.62614,3,1
14075,Dwyane Wade,0.020,0.52419,7,2
7312,Kobe Bryant,0.354,0.32777,4,3
13220,Derrick Rose,0.977,0.31651,1,4
13585,Kevin Durant,0.157,0.15893,5,5
12594,Dwight Howard,0.531,0.15287,2,6
10471,Rajon Rondo,0.004,0.10735,11,7
10006,Manu Ginóbili,0.017,0.10462,8,8
5722,Amar'e Stoudemire,0.007,0.06713,9,9
12981,Dirk Nowitzki,0.093,0.04621,6,10


**Building a XGBoost Model**

In [39]:
mvp_race_XGBoost = run_model(XGBRegressor(), X_train, X_test, Y, test)
mvp_race_XGBoost

,Player,Share,Predictions,Rank,Predicted_Rank
14075,Dwyane Wade,0.020,0.810279,7,1
14083,LeBron James,0.431,0.764443,3,2
13585,Kevin Durant,0.157,0.569759,5,3
13220,Derrick Rose,0.977,0.453084,1,4
7312,Kobe Bryant,0.354,0.448050,4,5
10006,Manu Ginóbili,0.017,0.099190,8,6
10471,Rajon Rondo,0.004,0.085248,11,7
12594,Dwight Howard,0.531,0.074594,2,8
10012,Tony Parker,0.002,0.055399,13,9
12981,Dirk Nowitzki,0.093,0.050845,6,10
